#### About
Camera projection matrix for mapping real world 3D cartesian coordinates to Image plane and vice versa.

1. Primary assumption is that all the following concepts are applicable for Pinhole camera.
2. x = PX is the equation to map a 2D pixel coordinate to 3D world coordinate where x is 2D pixel coordinate, P is projection matrix and X is 3D world coordinate.
3. Four coordinate systems that are involved include world, camera, image plane and sensor coordinate system.
4. Initially, The camera needs to be localised in the real world coordinate system.
5. The camera location is defined by the projection center followed by the rotational matrix. 
6. Projection matrix is comprised of 11 degrees of freedom - 6 degrees of freedom for camera extrinsic and 5 including distortion parameter for camera intrinsic.
7. It's described in the image below
![img.jpg](img.jpg)
8. We cannot easily invert Projection matrix as there is a loss of information while projecting 3D image to 2D.
9. Camera Extrinsic and Camera Intrinsic are parameter of a camera model. The Extrinsic describe the localisation of the camera in real world (Rotational and Translation).
10. Camera Intrinsic comprises of Camera constant(C- Distance of image plane to projection center), m - scale diff between x and y along with fx,fy(focal length), px,py(principal point) and distortion parameter(S- Shear parameter) describe how an image in 3D is mapped to 2D.
11. Camera Calibration is a technique used for calculating intrinsic parameter of the image.
